In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_list_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(36.35762598834591), np.float64(0.8545009828238505), np.float64(0.6860588906624249), np.float64(1.2639145173370645), np.float64(-30.90716389567639), np.float64(0.7199732022995978), np.float64(0.8765584908373518), np.float64(0.5413637674682404), np.float64(0.23993929546074183), np.float64(0.3676540157998898), np.float64(0.17093237568443084), np.float64(171.712164134283), np.float64(824290.2792509845), np.float64(2.9810343844826592)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=70)

cur node 0
optimal K 2
cur node 1
optimal K 6
cur node 2
optimal K 6
cur node 3
optimal K 2
cur node 4
optimal K 2
cur node 5
cur node 6
cur node 7
optimal K 2
cur node 8
optimal K 9
cur node 9
optimal K 2
cur node 10
optimal K 2
cur node 11
optimal K 2
cur node 12
optimal K 2
cur node 13
optimal K 4
cur node 14
optimal K 2
cur node 15
optimal K 4
cur node 16
cur node 17
optimal K 2
cur node 18
optimal K 2
cur node 19
cur node 20
optimal K 2
cur node 21
cur node 22
cur node 23
cur node 24
cur node 25
cur node 26
cur node 27
cur node 28
cur node 29
cur node 30
optimal K 2
cur node 31
optimal K 2
cur node 32
optimal K 2
cur node 33
optimal K 2
cur node 34
optimal K 2
cur node 35
cur node 36
optimal K 2
cur node 37
optimal K 2
cur node 38
cur node 39
cur node 40
cur node 41
cur node 42
cur node 43
cur node 44
cur node 45
cur node 46
cur node 47
cur node 48
cur node 49
cur node 50
cur node 51
cur node 52
cur node 53
cur node 54
optimal K 2
cur node 55
optimal K 3
cur node 56
optimal K 3
cu

In [9]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [10]:
seaching = test_query_generator()
with Timer():
    cur = test_tree.search(seaching)
print(cur.id)

Elapsed time: 0.001875099999779195 seconds
39


In [11]:
with Timer():
    res = test_tree.get_sorted_node_within_lv(test_tree.nodes[19].center, test_tree.nodes[1], lv=3)
print(res.keys())
print(res["sorted_cos_sim"])
print(res["euclide_dist"])
print([n.id for n in res["sorted_nodes"]])


generate cos sim, ecul dist, and nodes sorted by cos sim
Elapsed time: 0.0011542000002009445 seconds
dict_keys(['sorted_cos_sim', 'euclide_dist', 'sorted_nodes'])
[np.float64(-0.692381164354358), np.float64(-0.5957682990071481), np.float64(-0.5203709180975703), np.float64(-0.50415621287486), np.float64(-0.45528594593212984), np.float64(-0.4441102447385581), np.float64(-0.3931444919706754), np.float64(-0.3408978184417081), np.float64(-0.24420649358545185), np.float64(-0.23297574631837611), np.float64(-0.1901671681112758), np.float64(-0.09800860437144247), np.float64(0.07195162841033621), np.float64(0.12676534828510805), np.float64(0.170747981083863), np.float64(0.22363187080319194), np.float64(0.2738678998572105), np.float64(0.29677722327096256), np.float64(0.4703802879707088), np.float64(0.537921980009141), np.float64(0.5950050487226229), np.float64(0.6111899980469888), np.float64(1.0)]
[np.float64(1.0653642244759367), np.float64(1.0491737952990914), np.float64(0.927660377440605), np.f

In [12]:
a = test_tree.get_midle_graph_data(19, lv_degree=3)
print(a)
print([n.id for n in a["sorted_nodes"]])

reflection_node: 0 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
{'sorted_cos_sim': [np.float64(-0.9092899077346077), np.float64(-0.5770682441093181), np.float64(-0.5452817212112238), np.float64(-0.06417972150519574), np.float64(0.09161981914967166), np.float64(0.1362661537536247), np.float64(0.2832378608651501), np.float64(0.3506029407419899), np.float64(0.5657069202390209), np.float64(0.6217135544759613), np.float64(0.6805262815513853), np.float64(0.9166602500820263)], 'euclide_dist': [np.float64(1.311773804353229), np.float64(1.4273204431734674), np.float64(1.2404095593602367), np.float64(1.128014131963656), np.float64(1.0169128689651175), np.float64(1.3215302824840762), np.float64(1.0711422805238744), np.float64(0.9368323526779849), np.float64(0.7441162428152408), np.float64(0.698610140003174), np.float64(0.8914594055288028), np.float64(0.3409382094778211)], 'sorted_nodes': [<graph_tree_list_module.Node object at 0x00000236212E49D0>, <graph_tree_list_module.N

In [13]:
a = test_tree.lv_start
print(a)
print([a[i+1]-a[i] for i in range(len(a)-1)])

[0, 1, 3, 15, 44, 68, 85, 93]
[1, 2, 12, 29, 24, 17, 8]


In [14]:
test_tree.print_tree(test_tree.nodes[0])

└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 15
    │   │   │   ├── 44
    │   │   │   ├── 45
    │   │   │   ├── 46
    │   │   │   └── 47
    │   │   └── 16
    │   ├── 4
    │   │   ├── 17
    │   │   │   ├── 48
    │   │   │   └── 49
    │   │   └── 18
    │   │       ├── 50
    │   │       └── 51
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   │   ├── 19
    │   │   └── 20
    │   │       ├── 52
    │   │       └── 53
    │   └── 8
    │       ├── 21
    │       ├── 22
    │       ├── 23
    │       ├── 24
    │       ├── 25
    │       ├── 26
    │       ├── 27
    │       ├── 28
    │       └── 29
    └── 2
        ├── 9
        │   ├── 30
        │   │   ├── 54
        │   │   │   ├── 68
        │   │   │   │   ├── 85
        │   │   │   │   └── 86
        │   │   │   └── 69
        │   │   └── 55
        │   │       ├── 70
        │   │       ├── 71
        │   │       └── 72
        │   └── 31
        │       ├── 56
        │       │   ├── 73
        │       │   ├── 74
   

In [15]:
print(test_tree.nodes[0].center)

[0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ]


In [16]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), node=test_tree.nodes[11], degree=2)

AttributeError: 'graph_tree' object has no attribute 'reflection_search'

In [15]:
res

[[<graph_tree_list_module.Node at 0x24a0919b800>],
 array([0.45153535, 0.52339993, 0.68550112, 0.49472727, 0.76952637,
        0.561     , 0.06974494, 0.27134241, 0.13702421, 0.19445554,
        0.47458816, 0.49446303, 0.19104194, 0.73775   ]),
 [np.int64(934),
  np.int64(461),
  np.int64(767),
  np.int64(1807),
  np.int64(1065),
  np.int64(915),
  np.int64(107),
  np.int64(1006),
  np.int64(1318),
  np.int64(803)]]

In [16]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  181
hitted right result: 12
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(2): 20, np.int32(4): 20, np.int32(6): 14, np.int32(1): 12, np.int32(8): 9, np.int32(5): 7, np.int32(0): 6, np.int32(3): 6, np.int32(7): 5, np.int32(9): 1})
belongs to group:  2 

-- mine result100
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 22, np.int32(6): 11, np.int32(1): 11, np.int32(2): 11, np.int32(5): 10, np.int32(0): 10, np.int32(7): 8, np.int32(9): 7, np.int32(8): 7, np.int32(3): 3})
belongs to group:  4 

-- mine result200
search number:  2000
hitted right result: 100
-- kme

In [17]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[0.4695000904999095, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 0.4095001804998195, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07]
[0.46450013549986446, 0.40500020499979494, 0.37000020499979497, 0.41500020499979495, 0.40000020499979494, 0.43000020499979497, 0.3850002049997949, 0.41500020499979495, 0.42500020499979496, 0.4645001304998695]
0.8790082709917285
4.174001905998094
